In [ ]:
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import calendar;

In [ ]:

json_pages_info = {"pages": []}

options = Options()
options.add_argument("--headless") 
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

articles_count = 0

pages = 100

url = "https://www.ciperchile.cl/page/1/?s=Estallido+Social"

driver.get(url)

# Esperar a que la página cargue completamente
time.sleep(2)

with open(f"output.html", "w", encoding="utf-8") as f:
		f.write(driver.page_source)

for page in range(2, pages + 1):

	try:
		elements = driver.find_element(By.CLASS_NAME, "pb-4")
		articles = elements.find_elements(By.CLASS_NAME, "col-md-12")
	except Exception as e:
		print("Fatal error al encontrar los elementos de la página: ", e)
		break

	print(f"{len(articles)} articles found")

	for article in articles:
		print("Analyzing article...")

		date = article.find_element(By.CLASS_NAME, "article__text--date")
		print(date.text)
		originalDate = date.text[:10] if date else "not found"
		splitDate = originalDate.split("-")

		if(len(splitDate) != 3 or date == None):
			print("Fecha no encontrada")
			continue
		
		timestamp = calendar.timegm(time.strptime(f'{splitDate[2]}-{splitDate[1]}-{splitDate[0]} 00:00:00', '%Y-%m-%d %H:%M:%S'))

		# print("timestamp: " + str(timestamp))

		if (timestamp < 1573786800 or timestamp > 1702782000):
			print("La noticia no corresponde a la fecha solicitada")
			continue

		print(splitDate)

		try:
			link = article.find_elements(By.CLASS_NAME, "alticle-link")[1].get_attribute("href")
		except Exception as e:
			print("Error al obtener el enlace del artículo: ", e)
			continue

		try:
			category = article.find_element(By.CLASS_NAME, "alticle-link").text
			title = article.find_element(By.CLASS_NAME, "article__text--title").text
			description = article.find_element(By.CLASS_NAME, "article__text--epigraph").text

			link_info = {
				"newscast" : "Ciper",
				"title": title,
				"description": description,
				"category": category,
				"date": originalDate,
				"image_link": "The site does not provide a image",
				"author": "not found initially",
				"link": link,
			}

		except Exception as e:
			print("Error al obtener los datos del artículo: ", e)

			link_info = {
				"newscast": "Error",
				"title": "Error",
				"description": "Error",
				"category": "Error",
				"date": originalDate,
				"image_link": "Error",
				"author": "Error",
				"link": link
			}

		articles_count += 1
		print("Añadiendo articulo")
		json_pages_info["pages"].append(link_info)

	url = f"https://www.ciperchile.cl/page/{page}/?s=Estallido+Social"
	driver.get(url)
	time.sleep(2)

driver.quit()

with open(f"../archive/temp/ciper_pages.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)
